In [8]:
#Connecting to the API

from dataprep.connector import Connector,info
token = 'Z8oIDLe4PnO1pgoJwCQilw7vAGbk1VA3'
dc = Connector('./', _auth={'access_token':token})
dc

In [10]:
info('./')

,column_name,data_type
0,id,string
1,name,string
2,min_price,string
3,max_price,string
4,currency,string
5,url,string
6,sales_start_date,string
7,sales_end_date,string
8,event_date,string
9,event_time,string


Q1- What is the venue address of the 10 recent upcoming musical shows in British Columbia?

In [11]:
#API allows less than 1000 retrievals for each request so 999 used
#Queried based on classification name, which returns events with segment, genre, or subgenre Musical
#State symbol BC used for British columbia

df = await dc.query('events',classificationName="Musical",stateCode="BC",_count=999)

#values are sorted by event date and event time in ascending order and top 10 are returned
df=df.sort_values(by=["event_date","event_time"])
df[["name","event_date","event_time","genre_name","subgenre_name","address","city","state","country"]].iloc[0:10,]

,name,event_date,event_time,genre_name,subgenre_name,address,city,state,country
0,Come from Away,2021-03-18,20:00:00,[Theatre],[Musical],{'line1': '600 Block Hamilton St'},Vancouver,British Columbia,Canada
1,Come from Away,2021-03-19,20:00:00,[Theatre],[Musical],{'line1': '600 Block Hamilton St'},Vancouver,British Columbia,Canada
2,Come from Away,2021-03-20,14:00:00,[Theatre],[Musical],{'line1': '600 Block Hamilton St'},Vancouver,British Columbia,Canada
3,Come from Away,2021-03-20,20:00:00,[Theatre],[Musical],{'line1': '600 Block Hamilton St'},Vancouver,British Columbia,Canada
5,Come from Away,2021-03-21,14:00:00,[Theatre],[Musical],{'line1': '600 Block Hamilton St'},Vancouver,British Columbia,Canada
4,Come from Away,2021-03-21,19:30:00,[Theatre],[Musical],{'line1': '600 Block Hamilton St'},Vancouver,British Columbia,Canada
6,Menopause The Musical,2021-09-19,19:30:00,[Theatre],[Musical],{'line1': '2329 Crescent Way'},Abbotsford,British Columbia,Canada


Q2- What are the timings and venues for the events from 2021-02-2018 and before 2021-07-08 in New York state, United states?

In [12]:
#Queried using data ranges set by startDateTime and endDateTime parameters for the NY state

df = await dc.query('events',stateCode="NY",startDateTime="2021-02-18T12:00:00Z",endDateTime="2021-03-13T00:00:00Z",_count=999)

#values are sorted by event date and event time in ascending order and top 10 are returned
df=df.sort_values(by=["event_date","event_time"])
df[["name","event_date","event_time","address","city","state","country"]].iloc[0:10,]


,name,event_date,event_time,address,city,state,country
22,"Laughing Buddha Open Mic Spectacular, Jeff Law...",2021-02-18,17:30:00,{'line1': '487 Atlantic Avenue'},Brooklyn,New York,United States Of America
144,Open Mic Thursday - Laughing Buddha,2021-02-18,18:00:00,{'line1': '487 Atlantic Avenue'},Brooklyn,New York,United States Of America
6,Hadestown (NY),2021-02-18,19:00:00,{'line1': '219 West 48th Street'},New York,New York,United States Of America
15,Mean Girls (NY),2021-02-18,19:00:00,{'line1': '245 West 52nd Street'},New York,New York,United States Of America
18,"Emma Willmann, Jordan Rock, Janeane Garofalo",2021-02-18,19:30:00,{'line1': '487 Atlantic Avenue'},Brooklyn,New York,United States Of America
17,"Penthouse Comedy, Dina Hashem, Andy Fiori",2021-02-18,21:30:00,{'line1': '487 Atlantic Avenue'},Brooklyn,New York,United States Of America
21,"Laughing Buddha Open Mic Spectacular, Jeff Law...",2021-02-19,18:00:00,{'line1': '487 Atlantic Avenue'},Brooklyn,New York,United States Of America
149,Open Mic Friday - Laughing Buddha,2021-02-19,18:00:00,{'line1': '487 Atlantic Avenue'},Brooklyn,New York,United States Of America
14,Mean Girls (NY),2021-02-19,20:00:00,{'line1': '245 West 52nd Street'},New York,New York,United States Of America
16,Hadestown (NY),2021-02-19,20:00:00,{'line1': '219 West 48th Street'},New York,New York,United States Of America


Q3- What are the top 5 genres popular in British Columbia, Canada?

In [13]:


df = await dc.query('events',stateCode="BC",countryCode="CA",_count=999)
temp=list(df['genre_name'])
index=0

#convert object array format to stirngs to enable hasing by groupby function.
#shows with multiple genres are seperated by "/" in asecending order of strings
for i in temp:
    if (len(i)>1):
        temp[index]="/".join(sorted(list(set(i))))
    else:
        temp[index]=i[0]
    index+=1
df["genre_name"]=temp

#counts each type of genre and groups them together with its count
#values are sorted by count in ascending order and top 5 are returned
df_new=df.groupby(by="genre_name").size().reset_index(name="counts")
df_new=df_new.sort_values(by="counts",ascending=False)
df_new.iloc[0:5,]

,genre_name,counts
6,Fine Art,960
13,Rock,7
14,Theatre,7
3,Comedy,6
2,Children's Theatre,5


Q4- Are there any tickets available for comedy events in Canada on 2021-02-17 in the price range of [20,40] CAD?

In [14]:
#lower limit on sale date set by onsaleStartDateTime
#keyword comedy is used to search for all events with comedy in title, genre, segement, or subgenre

df = await dc.query('events',keyword="Comedy",countryCode="CA",onsaleStartDateTime="2021-02-17T22:00:00Z",_count=999)

#status onsale indicates if tickets are still available and the price range is checked before output

df.loc[(df.sale_status=="onsale") & (df.min_price>="20.0") & (df.max_price<="40.0")][["name","genre_name","min_price","max_price","sales_end_date","event_date","sale_status","city","country"]]


,name,genre_name,min_price,max_price,sales_end_date,event_date,sale_status,city,country
19,Pierre Hébert,[Comedy],22.0,22.0,2021-02-27T03:00:00Z,2021-02-26,onsale,Brossard,Canada
20,Pierre Hébert,[Comedy],20.0,20.0,2021-02-27T00:30:00Z,2021-02-26,onsale,Brossard,Canada
24,Simon Gouache,[Comedy],32.0,32.0,2021-03-05T02:00:00Z,2021-03-04,onsale,Brossard,Canada
51,Pierre-yves Roy Desmarais,[Comedy],25.0,25.0,2021-05-02T01:00:00Z,2021-05-01,onsale,Brossard,Canada
159,Jean Francois Mercier,[Comedy],38.0,38.0,2021-03-18T01:00:00Z,2021-03-17,onsale,Brossard,Canada
164,Melanie Couture,[Comedy],25.0,25.0,2021-05-08T01:00:00Z,2021-05-07,onsale,Brossard,Canada
246,Gala Comedie Star,[Theatre],39.0,39.0,2021-03-24T01:00:00Z,2021-03-23,onsale,Brossard,Canada


Q5- What are, on average, the top 10 most exepensive American cities for upcoming Sports events in 2021?

In [15]:
#keyword sports is used to get all sporting events in the US

df = await dc.query('events',keyword="sports",countryCode="US",_count=999)

#average price of each event is calculated by averaging the min and max posisble price a customer pays
df=df.astype({'max_price': 'float','min_price': 'float'})
df["average_price"]=(df["max_price"]+df["min_price"])/2

#data is grouped by states and cities using the mean of average price paid for all events in the city
df_new=df.groupby(["city","state"])["average_price"].mean().round(2).reset_index(name="average_cost")

#top 10 most expensive cities are returned
df_new.sort_values(by="average_cost",ascending=False).iloc[0:10,]

,city,state,average_cost
71,Miami,Florida,1588.75
66,Louisville,Kentucky,645.88
3,Atlanta,Georgia,508.50
5,Baltimore,Maryland,473.00
68,Madison,Wisconsin,458.75
27,Dallas,Texas,448.11
112,Tampa,Florida,426.40
12,Bronx,New York,422.19
19,Chicago,Illinois,335.00
25,Corpus Christi,Texas,310.00
